## Downloading IEMOCAP and extracting it

In [271]:
!pip install tensorflow-addons
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.28-7ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [272]:
from IPython.display import Audio, clear_output
from google.colab import files
import pandas as pd
import os
import numpy as np
import tensorflow_addons as tfa


files.upload()
!ls -lha kaggle.json
!pip install -q kaggle # Install kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d riccardopaolini/nlp-project-work
!unzip nlp-project-work.zip
clear_output()

folder = os.path.join(os.getcwd(), 'IEMOCAP')

conv_id = 0

In [273]:
df = []
for session in ['Session1','Session2','Session3','Session4','Session5']:
    session_path = os.path.join(folder, session)
    # 'dialogue' folder contains Emotions and Transcripts
    # 'sentences' folder contains Audios

    trans_folder = os.path.join(session_path, 'dialog', 'transcriptions')

    for trans_name in np.sort(os.listdir(trans_folder)):
        if trans_name[:2] != '._':
            emo_path = os.path.join(session_path, 'dialog', 'EmoEvaluation', trans_name)
            with open(os.path.join(trans_folder, trans_name), encoding='utf8') as trans_file, open(emo_path, encoding='utf8') as emo_file:
                conv_id += 1
                turn_id = 0
                for line in trans_file:
                    audio_name, text = line.split(':')
                    if trans_name.split('.')[0] in audio_name:
                        turn_id += 1

                        wav_path = os.path.join(session_path, 'sentences', 'wav', trans_name.split('.')[0], audio_name.split(' ')[0] + '.wav')

                        reached = False
                        count_em = {'Anger': 0, 'Happiness': 0, 'Sadness': 0, 'Neutral': 0, 'Frustration': 0, 'Excited': 0, 'Fear': 0, 'Surprise': 0, 'Disgust': 0, 'Other': 0}
                        for emo_line in emo_file:
                            if audio_name.split(' ')[0] in emo_line:
                                emotion, vad = emo_line.split('\t')[-2:]
                                vad = vad[1:-2].split(',')
                                reached = True
                            elif emo_line[0] == 'C' and reached:
                                evaluator = emo_line.split(':')[0]
                                emotions = emo_line.split(':')[1].split('(')[0].split(';')
                                emotions = [em.strip() for em in emotions]
                                for em in emotions:
                                    if em != '':
                                        count_em[em] += 1
                            elif reached:
                                emo_file.seek(0)
                                break
                                    

                        row = {'session_id': int(session[-1]),
                                'conv_id': conv_id, 
                                'turn_id': turn_id, 
                                'sentence': text.strip(),
                                'path': wav_path,
                                'emotion': emotion,
                                'valence': float(vad[0]),
                                'activation': float(vad[1]),
                                'dominance': float(vad[2])
                                }
                        
                        df.append(dict(**row, **count_em))

df = pd.DataFrame(df)

idx = np.array([os.path.exists(path) for path in df.path])
print(f'Missing Audios: {np.sum(~idx)}')
print('Missing Sentences:')
print(df.iloc[~idx,3])
df = df.iloc[idx, :]

Missing Audios: 48
Missing Sentences:
3854    [LAUGHTER], That's what they say.
3866                            Mmm, Hmm.
3880                                Yeah.
3898                               Kelly.
3915                           Yeah, man.
3939                      Uh-huh, uh-huh.
3961                              Uh-huh.
3968                                Yeah.
3972                                Yeah.
4010                       Well, I don't-
4044                                Yeah.
4827                        But, Listen--
4847                                Yeah.
4873                                Yeah.
4975                          We- I mean-
4991                                Okay.
5005                                Yeah.
5051                              Thanks.
5124                        to start off.
5181                                okay.
5192                                Okay.
5208                                Okay.
7893                                  

In [274]:
# selecting just first rows to avoid memory crash
df=df.head(1000)

## Setting Up

Before running this notebook, please ensure that you are on GPU runtime (`Runtime` > `Change runtime type` > `GPU`). The following cell will install [`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2) package & its dependencies.

## Model setup using `TFHub`

We will start by importing some libraries/modules.

In [275]:
import os

import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)

TF version: 2.12.0


First, we will download our model from TFHub & will wrap our model signature with [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) to be able to use this model like any other Keras layer. Fortunately, `hub.KerasLayer` can do both in just 1 line.

**Note:** When loading model with `hub.KerasLayer`, model becomes a bit opaque but sometimes we need finer controls over the model, then we can load the model with `tf.keras.models.load_model(...)`.

In [276]:
pretrained_layer = hub.KerasLayer("https://tfhub.dev/vasudevgupta7/wav2vec2/1", trainable=True)

You can refer to this [script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/export2hub.py) in case you are interested in the model exporting script. Object `pretrained_layer` is the freezed version of [`Wav2Vec2Model`](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/wav2vec2/modeling.py). These pre-trained weights were converted from HuggingFace PyTorch [pre-trained weights](https://huggingface.co/facebook/wav2vec2-base) using [this script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/convert_torch_to_tf.py).

Originally, wav2vec2 was pre-trained with a masked language modelling approach with the objective to identify the true quantized latent speech representation for a masked time step. You can read more about the training objective in the paper- [wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations](https://arxiv.org/abs/2006.11477).

Now, we will be defining a few constants and hyper-parameters which will be useful in the next few cells. `AUDIO_MAXLEN` is intentionally set to `246000` as the model signature only accepts static sequence length of `246000`.

In [277]:
AUDIO_MAXLEN = 246000
LABEL_MAXLEN = 11
BATCH_SIZE = 2

In the following cell, we will wrap `pretrained_layer` & a dense layer (LM head) with the [Keras's Functional API](https://www.tensorflow.org/guide/keras/functional).

In [278]:
from tensorflow.keras.layers import Input, Dense, Dropout
IEMOCAP_CLASSES = LABEL_MAXLEN
#change output classes config.vocab_size

inputs = tf.keras.Input(shape=(AUDIO_MAXLEN,))
hidden_states = pretrained_layer(inputs)
out = tf.keras.layers.GlobalMaxPool1D()(hidden_states)
out = Dense(128, activation='relu')(out)
out = Dropout(0.1)(out)
out = Dense(32, activation = 'relu')(out)
outputs = Dense(IEMOCAP_CLASSES, activation = 'Softmax')(out)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

The dense layer (defined above) is having an output dimension of `vocab_size` as we want to predict probabilities of each token in the vocabulary at each time step.

## Setting up training state

In TensorFlow, model weights are built only when `model.call` or `model.build` is called for the first time, so the following cell will build the model weights for us. Further, we will be running `model.summary()` to check the total number of trainable parameters.

In [279]:
model(tf.random.uniform(shape=(BATCH_SIZE, AUDIO_MAXLEN)))
model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 246000)]          0         
                                                                 
 keras_layer_9 (KerasLayer)  (None, 768, 768)          94371712  
                                                                 
 global_max_pooling1d_7 (Glo  (None, 768)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_33 (Dense)            (None, 128)               98432     
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_34 (Dense)            (None, 32)                4128      
                                                          

Now, we need to define the `loss_fn` and optimizer to be able to train the model. The following cell will do that for us. We will be using the `Adam` optimizer for simplicity. `CTCLoss` is a common loss type that is used for tasks (like `ASR`) where input sub-parts can't be easily aligned with output sub-parts. You can read more about CTC-loss from this amazing [blog post](https://distill.pub/2017/ctc/).


`CTCLoss` (from [`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2) package) accepts 3 arguments: `config`, `model_input_shape` & `division_factor`. If `division_factor=1`, then loss will simply get summed, so pass `division_factor` accordingly to get mean over batch.

In [280]:
from tensorflow.keras.losses import CategoricalCrossentropy

loss_fn = 'categorical_crossentropy',

LEARNING_RATE = 5e-5

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

## Loading & Pre-processing data

Let's now download the LibriSpeech dataset from the [official website](http://www.openslr.org/12) and set it up.

In [281]:
flac_files = df.iloc[:,4].to_list()
flac_files[:4]


['/content/IEMOCAP/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_F000.wav',
 '/content/IEMOCAP/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_M000.wav',
 '/content/IEMOCAP/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_F001.wav',
 '/content/IEMOCAP/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_M001.wav']

Alright, so each sub-directory has many `.flac` files and a `.txt` file. The `.txt` file contains text transcriptions for all the speech samples (i.e. `.flac` files) present in that sub-directory.

Similarly, we will define a function for loading a speech sample from a `.flac` file.

`REQUIRED_SAMPLE_RATE` is set to `16000` as wav2vec2 was pre-trained with `16K` frequency and it's recommended to fine-tune it without any major change in data distribution due to frequency.

In [282]:
import soundfile as sf

REQUIRED_SAMPLE_RATE = 16000

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".wav")]
  return audio

Now, we will pick some random samples & will try to visualize them.

In [283]:
from IPython.display import Audio
import random

file_id = random.choice([f[:-len(".wav")] for f in flac_files])
flac_file_path = os.path.join(f"{file_id}.wav")

Now, we will combine all the speech & text samples and will define the function (in next cell) for that purpose.

In [284]:
def fetch_sound_text_mapping(flac_paths, emotions):

  speech_samples = []
  for f in flac_paths:
    speech_samples.append(read_flac_file(f))
  
  assert len(emotions) == len(speech_samples)

  samples = [(speech_samples[file_id], emotions[file_id]) for file_id in range(len(emotions)) if len(speech_samples[file_id]) < AUDIO_MAXLEN]

  return samples
  


It's time to have a look at a few samples ...

In [285]:
samples = fetch_sound_text_mapping(flac_files, df.emotion.to_list())
samples[:5]

[(array([-0.0050354 , -0.00497437, -0.0038147 , ..., -0.00265503,
         -0.00317383, -0.00418091]),
  'neu'),
 (array([-0.00354004, -0.00308228, -0.0062561 , ...,  0.00341797,
          0.00274658,  0.00256348]),
  'fru'),
 (array([ 0.00094604, -0.00094604, -0.0007019 , ..., -0.00045776,
         -0.00033569, -0.00128174]),
  'neu'),
 (array([ 0.00021362, -0.00048828, -0.00143433, ..., -0.00378418,
         -0.00375366, -0.00292969]),
  'fru'),
 (array([-0.00036621, -0.00015259,  0.00042725, ..., -0.00030518,
         -0.00018311,  0.00088501]),
  'neu')]

Note: We are loading this data into memory as we working with a small amount of dataset in this notebook. But for training on the complete dataset (~300 GBs), you will have to load data lazily. You can refer to [this script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/data_utils.py) to know more on that.

Let's pre-process the data now !!!

We will first define the tokenizer & processor using `gsoc-wav2vec2` package. Then, we will do very simple pre-processing. `processor` will normalize raw speech w.r.to frames axis and `tokenizer` will convert our model outputs into the string (using the defined vocabulary) & will take care of the removal of special tokens (depending on your tokenizer configuration).

In [286]:
from wav2vec2 import Wav2Vec2Processor
from tensorflow.keras.utils import to_categorical

#tokenizer = Wav2Vec2Processor(is_tokenizer=True)
processor = Wav2Vec2Processor(is_tokenizer=False)

def preprocess_emotions(emotion):
  # tokenize function for emotions
  encoded_dict= {'ang':0, 'dis':1, 'exc':2, 'fea':3, 'fru':4, 'hap':5,'neu':6, 'oth':7, 'sad':8, 'sur':9, 'xxx':10}
  label=to_categorical(encoded_dict.get(emotion), num_classes=11)
  return tf.constant(label, dtype=tf.int32)

def preprocess_speech(audio):
  # tokenize function for audio
  audio = tf.constant(audio, dtype=tf.float32)
  return processor(tf.transpose(audio))


In [287]:
print(preprocess_emotions('ang'))

tf.Tensor([1 0 0 0 0 0 0 0 0 0 0], shape=(11,), dtype=int32)


Now, we will define the python generator to call the preprocessing functions we defined in above cells.

In [288]:
## emotions instead of text, change samples
def inputs_generator():
  for speech, emotion in samples:
    yield preprocess_speech(speech), preprocess_emotions(emotion)

## Setting up `tf.data.Dataset`

Following cell will setup `tf.data.Dataset` object using its `.from_generator(...)` method. We will be using the `generator` object, we defined in the above cell.

**Note:** For distributed training (especially on TPUs), `.from_generator(...)` doesn't work currently and it is recommended to train on data stored in `.tfrecord` format (Note: The TFRecords should ideally be stored inside a GCS Bucket in order for the TPUs to work to the fullest extent).

You can refer to [this script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/make_tfrecords.py) for more details on how to convert LibriSpeech data into tfrecords.

In [289]:
output_signature = (
    tf.TensorSpec(shape=(None),  dtype=tf.float32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
)

dataset = tf.data.Dataset.from_generator(inputs_generator, output_signature=output_signature)

In [290]:
BUFFER_SIZE = len(flac_files)
SEED = 42

dataset = dataset.shuffle(BUFFER_SIZE, seed=SEED)

We will pass the dataset into multiple batches, so let's prepare batches in the following cell. Now, all the sequences in a batch should be padded to a constant length. We will use the`.padded_batch(...)` method for that purpose.

In [291]:
dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=(AUDIO_MAXLEN, LABEL_MAXLEN), padding_values=(0.0, 0))

Accelerators (like GPUs/TPUs) are very fast and often data-loading (& pre-processing) becomes the bottleneck during training as the data-loading part happens on CPUs. This can increase the training time significantly especially when there is a lot of online pre-processing involved or data is streamed online from GCS buckets. To handle those issues, `tf.data.Dataset` offers the `.prefetch(...)` method. This method helps in preparing the next few batches in parallel (on CPUs) while the model is making predictions (on GPUs/TPUs) on the current batch.

In [292]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

Since this notebook is made for demonstration purposes, we will be taking first `num_train_batches` and will perform training over only that. You are encouraged to train on the whole dataset though. Similarly, we will evaluate only `num_val_batches`.

In [293]:
num_train_batches = 400 # x batch size (2)
num_val_batches = 100 # x batch size (2)

train_dataset = dataset.take(num_train_batches)
val_dataset = dataset.skip(num_train_batches).take(num_val_batches)

## Model training

For training our model, we will be directly calling `.fit(...)` method after compiling our model with `.compile(...)`.

In [294]:
f1 = tfa.metrics.F1Score(num_classes=IEMOCAP_CLASSES,average='weighted')
accuracy = 'accuracy'


model.compile(optimizer, loss=loss_fn, metrics = [accuracy,f1])

In [295]:
import gc

del(df)
gc.collect()

2245

The above cell will set up our training state. Now we can initiate training with the `.fit(...)` method.

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)
history.history

Epoch 1/3


400/400 [==============================] - 339s 715ms/step - loss: 2.0686 - accuracy: 0.2175 - f1_score: 0.1667 - val_loss: 1.9633 - val_accuracy: 0.2649 - val_f1_score: 0.1109
Epoch 2/3
400/400 [==============================] - ETA: 0s - loss: 1.9845 - accuracy: 0.2138 - f1_score: 0.1543

In [ ]:
train_dataset

In [ ]:
#print(preprocess_speech(samples[0][0]))
#model.predict(preprocess_speech(samples[0][0]))

Let's save our model with `.save(...)` method to be able to perform inference later. You can also export this SavedModel to TFHub by following [TFHub documentation](https://www.tensorflow.org/hub/publish).

In [ ]:
save_dir = "finetuned-wav2vec2"
model.save(save_dir, include_optimizer=False)

Note: We are setting `include_optimizer=False` as we want to use this model for inference only.

## Evaluation

Now we will be computing Word Error Rate over the validation dataset

**Word error rate** (WER) is a common metric for measuring the performance of an automatic speech recognition system. The WER is derived from the Levenshtein distance, working at the word level. Word error rate can then be computed as: WER = (S + D + I) / N = (S + D + I) / (S + D + C) where S is the number of substitutions, D is the number of deletions, I is the number of insertions, C is the number of correct words, N is the number of words in the reference (N=S+D+C). This value indicates the percentage of words that were incorrectly predicted. 

You can refer to [this paper](https://www.isca-speech.org/archive_v0/interspeech_2004/i04_2765.html) to learn more about WER.

We will use `load_metric(...)` function from [HuggingFace datasets](https://huggingface.co/docs/datasets/) library. Let's first install the `datasets` library using `pip` and then define the `metric` object.

In [ ]:
#!pip3 install -q datasets

#from datasets import load_metric
#metric = load_metric("wer")

In [ ]:
@tf.function(jit_compile=True)
def eval_fwd(batch):
  logits = model(batch, training=False)
  return logits

It's time to run the evaluation on validation data now.

In [ ]:
from tqdm.auto import tqdm

for speech, labels in tqdm(val_dataset, total=num_val_batches):
    predictions  = eval_fwd(speech)
    print(predictions)
    #predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
    #references = [tokenizer.decode(label, group_tokens=False) for label in labels.numpy().tolist()]
    #metric.add_batch(references=references, predictions=predictions)

We are using the `tokenizer.decode(...)` method for decoding our predictions and labels back into the text and will add them to the metric for `WER` computation later.

Now, let's calculate the metric value in following cell:

In [ ]:
metric.compute()

**Note:** Here metric value doesn't make any sense as the model is trained on very small data and ASR-like tasks often require a large amount of data to learn a mapping from speech to text. You should probably train on large data to get some good results. This notebook gives you a template to fine-tune a pre-trained speech model.

## Inference

Now that we are satisfied with the training process & have saved the model in `save_dir`, we will see how this model can be used for inference.

First, we will load our model using `tf.keras.models.load_model(...)`.

In [ ]:
finetuned_model = tf.keras.models.load_model(save_dir)

Let's download some speech samples for performing inference. You can replace the following sample with your speech sample also.

In [ ]:
!wget https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/SA2.wav

Now, we will read the speech sample using `soundfile.read(...)` and pad it to `AUDIO_MAXLEN` to satisfy the model signature. Then we will normalize that speech sample using the `Wav2Vec2Processor` instance & will feed it into the model.

In [ ]:
import numpy as np

speech, _ = sf.read("SA2.wav")
speech = np.pad(speech, (0, AUDIO_MAXLEN - len(speech)))
speech = tf.expand_dims(processor(tf.constant(speech)), 0)

outputs = finetuned_model(speech)
outputs

Let's decode numbers back into text sequence using the `Wav2Vec2tokenizer` instance, we defined above.

In [ ]:
predictions = tf.argmax(outputs, axis=-1)
predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
predictions

This prediction is quite random as the model was never trained on large data in this notebook (as this notebook is not meant for doing complete training). You will get good predictions if you train this model on complete LibriSpeech dataset.

Finally, we have reached an end to this notebook. But it's not an end of learning TensorFlow for speech-related tasks, this [repository](https://github.com/tulasiram58827/TTS_TFLite) contains some more amazing tutorials. In case you encountered any bug in this notebook, please create an issue [here](https://github.com/vasudevgupta7/gsoc-wav2vec2/issues).

In [ ]:
read_flac_file('SA2.wav')
Audio(filename='SA2.wav')